In [83]:
import requests
from urllib.parse import urlencode

In [52]:
#Assign your Genius.com credentials and select your artist
import lyricsgenius as genius
geniusCreds = ""
api = genius.Genius(geniusCreds, skip_non_songs=True)

In [41]:
from fuzzywuzzy import process
from fuzzywuzzy import utils

In [104]:
import spotipy
import spotipy.util as spotipy_util
scope = 'user-library-read'
username = 'blasternt'
spotify_token = spotipy_util.prompt_for_user_token(username,scope,client_id='',client_secret='',redirect_uri='https://chen-ye.github.io/lyricize/')
if spotify_token:
    spotify = spotipy.Spotify(auth=spotify_token)
else:
    print("Can't get token for", username)

In [105]:
playlist = spotify.user_playlist(username, '0BSdjrucaGOW2uqYLvodj4', fields='tracks,next')

In [106]:
track_items = []
results = playlist['tracks']
while results:
    track_items = track_items + results['items']
    if results['next']:
        results = spotify.next(results)
    else:
        results = None


In [107]:
songs = [(track_result['track']['name'], track_result['track']['artists'][0]['name']) for track_result in track_items]

In [117]:
songs_strings = [title + ',' + artist for (title, artist) in songs]

In [ ]:
song_results = [ api.search_song(song[0], artist=song[1], get_full_info=False) for song in songs ]

In [46]:
song_lyrics_processed = { song:utils.full_process(song.lyrics, force_ascii=True) if song else "" for song in song_results }

In [ ]:
process.extract("please don't", song_lyrics_processed)

In [76]:
genius_results = api.search_genius_web("please don't")

In [77]:
genius_lyric_results = genius_results['sections'][2]['hits']

In [78]:
[(result['result']['title'], result['result']['primary_artist']['name']) for result in genius_lyric_results]

[('Fuck Love', 'XXXTENTACION'),
 ('Wet Dreamz', 'J. Cole'),
 ('Heathens', '\u200btwenty one pilots'),
 ('Marvin’s Room', 'Drake'),
 ('The Worst', 'Jhené Aiko')]

In [109]:
def search_lyrics_page(query, page=1):
    """Use the web-version of Genius search"""
    endpoint = "search/lyric?"
    params = {'page': page, 'q': query}

    # This endpoint is not part of the API, requires different formatting
    url = "https://genius.com/api/" + endpoint + urlencode(params)
    response = requests.get(url)
    return response.json()['response'] if response else None

In [118]:
def search_lyrics_in_library(query):
    next_page = 1
    while next_page:
        genius_results = search_lyrics(query, next_page)
        genius_lyric_results = genius_results['sections'][0]['hits']
        song_results = [(result['result']['title'], result['result']['primary_artist']['name']) for result in genius_lyric_results]
        print(song_results)
        song_results_filtered = [result for result in genius_lyric_results if (result['result']['title'] + ',' + result['result']['primary_artist']['name'] in songs_strings) ]
        if song_results_filtered:
            print([(result['result']['title'], result['result']['primary_artist']['name'], result) for result in song_results_filtered])
            break
        next_page = genius_results['next_page']

In [119]:
search_lyrics_in_library("gasping")

[('The Great Gatsby (Chapter III)', 'F. Scott Fitzgerald'), ('The Great Gatsby (Chapter VII)', 'F. Scott Fitzgerald'), ('Black Widow', 'Iggy Azalea'), ('Obsessed', 'Mariah Carey'), ('Lord of the Flies (Chap. 1: The Sound of the Shell)', 'William Golding'), ('Trill Luv', 'Yakki Divioshi'), ('Yaphet Kotto', 'Childish Gambino'), ('Helplessly Hoping', 'Crosby, Stills & Nash'), ('Lord of the Flies (Chap. 2: Fire on the Mountain)', 'William Golding'), ('Guillotine (Swordz)', 'Raekwon')]
[('BTS - DNA (English Translation)', 'Genius English Translations'), ('Bring It On', 'JAY-Z'), ('The Type', 'Sarah Kay'), ('Rap God (Remix)', 'K.A.A.N.'), ('We All Die One Day', 'Obie Trice'), ('Sic Transit Gloria... Glory Fades', 'Brand New'), ('Gang Bang Anyway', 'The Game'), ('Lost', 'Illenium'), ('Day of the Dead', 'Hollywood Undead'), ('French Kiss', 'Trey Songz')]
[('Let’s Begin', 'Jon Bellion'), ('God’s Plan', 'Phora'), ('A Christmas Carol (Part 3)', 'Charles Dickens'), ('Fat Raps (Remix)', 'Big Sean')